In [10]:
import os
import pandas as pd
import numpy as np
from mistralai import Mistral
import time
import json


In [11]:
data = pd.read_csv("sanitized_tweets_sentiment.csv", on_bad_lines='skip')

In [12]:
def empty_complaints():
    return "[]"

def reset_discomfort_score():
    return 0

data["complaints"] = data["complaints"].apply(empty_complaints)
data["discomfort_score"] = data["discomfort_score"].apply(reset_discomfort_score)
data

,Unnamed: 0,id,screen_name,name,created_at,full_text,complaints,discomfort_score,month,sentiment
0,0,172517,gptournier,Guillaume Tournier,2023-11-16 15:13:18+00:00,@engiepartfr \n6 mois d’attente et tjs aucune ...,[],0,2023-11,Négatif
1,1,172868,jouanetwan,Jouan Etwan 💚💛,2023-11-26 07:34:34+00:00,"bonjour @engiepartsav , l’appli #monpilotageel...",[],0,2023-11,Négatif
2,2,173784,vince_thouvenin,thouvenin,2023-12-21 14:27:08+00:00,@engiepartfr mon syndic de copropriété sergic ...,[],0,2023-12,Négatif
3,3,174049,BiduleAnatheme,Anathème Bidule,2023-12-28 21:32:58+00:00,@engiepartsav vous envisagez de vous occuper d...,[],0,2023-12,Négatif
4,4,174068,vince_thouvenin,thouvenin,2023-12-29 10:08:10+00:00,"@engiepartsav retour de votre technicien ""vous...",[],0,2023-12,Négatif
...,...,...,...,...,...,...,...,...,...,...
580,571,189658,djofthp,jojodamido,2025-03-03 15:24:10+00:00,@engiepartfr on vous parle,[],0,2025-03,Négatif
581,572,189662,julien_ducerf,Julien Ducerf,2025-03-03 17:50:42+00:00,@engiepartfr c'est du harcèlement pour augment...,[],0,2025-03,Négatif
582,573,189663,SouareFirst,So First,2025-03-03 18:47:06+00:00,@engiepartfr bonjour engie les voleurs ! atten...,[],0,2025-03,Négatif
583,574,189670,Leoletonneau,Tigre & Dragon 🐯 🐉,2025-03-03 23:25:45+00:00,"@engiepartfr vraiment des escrocs, des montant...",[],0,2025-03,Négatif


In [ ]:
data.to_csv("sanitized_tweets_without_complaints_type.csv", index=False)

In [13]:
api_key = "avgxRmy1TsKOnsZXh75nuie5jDizDEEr"
model = "mistral-large-latest"
agent_id = "ag:e59f61ca:20250311:categorisation-de-problemes:19714d72"

client = Mistral(api_key=api_key)

In [6]:
chat_response = client.agents.complete(
                agent_id=agent_id,
                max_tokens=131072,
                response_format={
                    "type": "json_object",
                    },
                messages=[{"role": "user", "content": data.loc[0].to_json()}]
            )

print(chat_response.choices[0].message.content)

 {"Unnamed: 0":"0","id":"172517","screen_name":"gptournier","name":"Guillaume Tournier","created_at":"2023-11-16 15:13:18+00:00","full_text":"@engiepartfr \\n6 mois d\u2019attente et tjs aucune r\u00e9ponse de l\u2019agence engie home service grand marseille \u00e0 aubagne sur un devis, malgr\u00e9 des multiples relances. je reste coinc\u00e9 \u00e0 appeler la plate-forme t\u00e9l et \u00e0 entendre qu\u2019on me rappelle\\npour le num\u00e9ro 1 du service client c\u2019est dommage","complaints":"['service client injoignable', 'délai d’intervention']","discomfort_score":80,"month":"2023-11","sentiment":"N\u00e9gatif"}


In [27]:
json_data_categorized = []

In [28]:
for i in range(len(data.index)):
    while True:
        try:
            chat_response = client.agents.complete(
                        agent_id=agent_id,
                        max_tokens=131072,
                        response_format={
                            "type": "json_object",
                            },
                        messages=[{"role": "user", "content": data.loc[i].to_json()}]
                    )

            json_data_categorized.append(json.loads(chat_response.choices[0].message.content))
            time.sleep(2)
            if i % 100 == 0:
                print(f"Tweet {i} processed")

            break


        except Exception as e:
            if "429" in str(e):  # rate limit
                print("Rate limit atteinte. Pause...")
                time.sleep(120)
            else:
                print("Autre erreur :", e)
                break

Tweet 0 processed
Rate limit atteinte. Pause...
Autre erreur : Expecting value: line 1 column 205 (char 204)
Autre erreur : Expecting value: line 1 column 205 (char 204)
Rate limit atteinte. Pause...
Rate limit atteinte. Pause...
Autre erreur : Expecting value: line 1 column 225 (char 224)
Autre erreur : Expecting value: line 1 column 205 (char 204)
Autre erreur : Expecting value: line 1 column 205 (char 204)
Autre erreur : Expecting value: line 1 column 205 (char 204)
Autre erreur : Expecting value: line 1 column 205 (char 204)
Autre erreur : Expecting value: line 1 column 208 (char 207)
Autre erreur : Expecting value: line 1 column 205 (char 204)
Autre erreur : Expecting value: line 1 column 205 (char 204)
Autre erreur : Expecting value: line 1 column 205 (char 204)
Autre erreur : Expecting value: line 1 column 205 (char 204)
Tweet 100 processed
Autre erreur : Expecting value: line 1 column 205 (char 204)
Autre erreur : Expecting value: line 1 column 205 (char 204)
Autre erreur : Exp

In [ ]:
json_better_categorisation = json.dumps(json_data_categorized, indent=4)
with open("better_categorisation.json", "w") as f:
    f.write(json_better_categorisation)
    f.close()
new_df = pd.read_json("better_categorisation.json")
new_df = pd.DataFrame(json_data_categorized)
new_df.to_csv("better_categorisation.csv", index=False)


ValueError: Invalid file path or buffer object type: <class 'list'>

In [26]:
print(len(json_data_categorized))


print(json_data_categorized[10])

11
{'Unnamed: 0': '0', 'id': '172517', 'screen_name': 'gptournier', 'name': 'Guillaume Tournier', 'created_at': '2023-11-16 15:13:18+00:00', 'full_text': '@engiepartfr \n6 mois d’attente et tjs aucune réponse de l’agence engie home service grand marseille à aubagne sur un devis, malgré des multiples relances. je reste coincé à appeler la plate-forme tél et à entendre qu’on me rappelle\npour le numéro 1 du service client c’est dommage', 'complaints': "['service client injoignable', 'délai d’intervention']", 'discomfort_score': 90, 'month': '2023-11', 'sentiment': 'Négatif'}
